<a href="https://colab.research.google.com/github/bamirandab/bry/blob/master/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bamirandab","key":"791f88a026a8d05c7b1856405ea5fc1e"}'}

In [ ]:
!pip install pyGAM

In [ ]:
import pandas as pd
import numpy as np
import math

from sklearn.linear_model import LinearRegression, Lasso
import pygam
from sklearn.metrics import mean_squared_error,confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
# !pip install -q kaggle

# ! mkdir "/content/drive/MyDrive/Kaggle compentences"

# ! cp kaggle.json "/content/drive/MyDrive/Kaggle compentences"


#  ! chmod 600 "/content/drive/MyDrive/Kaggle compentences/kaggle.json"

#  
# !kaggle competitions download -c titanic > "/content/drive/MyDrive/Kaggle compentences/Titanic"

# !cp gender_submission.csv test.csv train.csv "/content/drive/MyDrive/Kaggle compentences/Titanic"

In [ ]:
path = '/content/drive/MyDrive/Kaggle compentences/Titanic'

train_path = '{}/{}'.format(path,'train.csv')
test_path = '{}/{}'.format(path,'test.csv')
gender = '{}/{}'.format(path,'gender_submission.csv')

In [ ]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
display(train_df.head())
print(train_df.shape)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


(891, 12)


In [ ]:
train_df.SibSp.value_counts()

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

In [ ]:
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
def cabin_format(x):
  try:
    if math.isnan(x) == True:
      temp = 'UNDIFINED'
    else:
      temp = str(x)[0]
  except:
    temp = str(x)[0]
  return temp

def format_df_model(df):
  df['gender'] = df.Sex.apply(lambda x: 1 if x=='male' else 0)
  df['Cabin_letter'] = df.Cabin.apply(cabin_format)
  return df

train_df.Cabin.apply(cabin_format).value_counts()

UNDIFINED    687
C             59
B             47
D             33
E             32
A             15
F             13
G              4
T              1
Name: Cabin, dtype: int64

In [ ]:
train = format_df_model(train_df)

In [ ]:
train_age = train[["Age","Pclass","gender","SibSp",	"Parch","Embarked","Cabin_letter"]].copy()
train_age.Embarked.fillna('C',inplace=True)
one_hot_columns=["Pclass","Embarked","Cabin_letter"]
train_age = pd.get_dummies(train_age,columns=one_hot_columns)
train_age_na = train_age.dropna()

train_age_columns = train_age_na.drop("Age",axis=1).columns

In [ ]:
age_model_lr = LinearRegression()
age_model_lss = Lasso(random_state=42)
age_model_gamlss = pygam.LinearGAM()
age_model_rf = RandomForestRegressor(random_state=42,n_estimators=20)

In [ ]:

age_model_lr.fit(train_age_na.drop("Age",axis=1),train_age_na.Age)
age_model_lss.fit(train_age_na.drop("Age",axis=1),train_age_na.Age)
age_model_gamlss.fit(train_age_na.drop("Age",axis=1),train_age_na.Age)
age_model_rf.fit(train_age_na.drop("Age",axis=1),train_age_na.Age)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=20, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
mean_squared_error(
    age_model_lr.predict(train_age_na.drop("Age",axis=1)),
    train_age_na.Age)

156.4372637758228

In [ ]:
mean_squared_error(
    age_model_lss.predict(train_age_na.drop("Age",axis=1)),
    train_age_na.Age)

170.27735207527144

In [ ]:
mean_squared_error(
    age_model_gamlss.predict(train_age_na.drop("Age",axis=1)),
    train_age_na.Age)

142.62869092489802

In [ ]:

mean_squared_error(
    age_model_rf.predict(train_age_na.drop("Age",axis=1)),
    train_age_na.Age)

107.23296918208835

In [ ]:
base_predict = train_age[train_age.Age.isna()]
# base_predict
predict_age = age_model_rf.predict(base_predict.drop("Age",axis=1)).astype(int)
predict_age

array([35, 34, 15, 27, 20, 28, 28, 20, 27, 27, 28, 32, 20, 22, 49, 43, 17,
       28, 28, 20, 28, 28, 28, 28, 20, 28, 35,  9, 16, 28, 28,  9, 22, 46,
        3, 12, 25, 43, 20, 35, 20,  9, 32, 28,  5, 25, 14, 20, 28, 35, 35,
       20, 46, 20, 34, 43, 43, 49, 20, 31, 39, 28, 35,  9, 20, 39, 28, 27,
       49, 27, 20, 20, 32, 15, 20, 47, 28, 35,  5, 28, 35, 34, 25, 27, 28,
       35, 27, 28, 25, 28, 55, 35, 28, 34, 35, 28, 43, 34,  5, 25, 27, 28,
       20, 46, 28, 35, 27, 27, 49, 27, 16, 28, 25, 35, 43, 35, 28, 25, 27,
       20, 14, 27, 28, 25, 35, 27, 28, 46, 28, 20, 35, 35, 46, 25, 28, 28,
       28, 20, 28, 28, 31, 34, 20, 28, 20, 17, 49, 35, 20, 34, 28, 28, 47,
       28, 43, 32, 27, 34, 35, 13, 35, 12, 43, 44, 35, 28, 35, 27, 28, 29,
        9, 24, 27, 12, 28, 28, 22])

In [ ]:
def age_format(df):
  train_columns=train_age_columns
  bool_age_na = df.Age.isna()
  df_dummies = pd.get_dummies(df,columns=one_hot_columns)
  df[df_dummies.columns] = df_dummies
  Age_predict = pd.Series(age_model_rf.predict(df[train_columns]))
  df["Age"] = pd.Series( np.multiply(df["Age"].fillna(0), ~bool_age_na) + np.multiply(Age_predict, bool_age_na) )

def women_and_childrens_flag(df):
  df["flag_women_childrens"] = (df.gender == 0) & ( df.Age <= 10 )
  result = df["flag_women_childrens"].astype(int)
  df["flag_women_childrens"] = result

def format_df(df):
  df['gender'] = df.Sex.apply(lambda x: 1 if x=='male' else 0)
  df['Cabin_letter'] = train.Cabin.apply(cabin_format)
  df.Embarked.fillna("C",inplace=True)
  age_format(df)
  women_and_childrens_flag(df)


In [ ]:
format_df(train_df)

test = test_df.copy()

format_df(test)

In [ ]:
standard_columns = ["Age","Fare"]
model_standard = StandardScaler()
standard_df = pd.concat([ train_df[standard_columns], test[standard_columns] ],axis=0).reset_index(drop=True)
model_standard.fit(standard_df)

array([[-0.56586146, -0.50340247],
       [ 0.61628515,  0.73422195],
       [-0.27032481, -0.49035618],
       ...,
       [ 0.65322723, -0.50340247],
       [-0.06094781, -0.48794021],
       [-0.88504105, -0.21139189]])

In [ ]:
def scaler(df,standard_columns = ["Age","Fare"] ):
  df[standard_columns] = model_standard.transform(df[standard_columns])

In [ ]:
scaler(train_df)

scaler(test)

In [ ]:
train_age_columns

Index(['gender', 'SibSp', 'Parch', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Cabin_letter_A',
       'Cabin_letter_B', 'Cabin_letter_C', 'Cabin_letter_D', 'Cabin_letter_E',
       'Cabin_letter_F', 'Cabin_letter_G', 'Cabin_letter_T',
       'Cabin_letter_UNDIFINED'],
      dtype='object')

In [ ]:
train_df.Cabin_letter.value_counts()

UNDIFINED    687
C             59
B             47
D             33
E             32
A             15
F             13
G              4
T              1
Name: Cabin_letter, dtype: int64

In [ ]:
train_final_columns = ['Survived', 'Pclass', 'Age', 'SibSp',
       'Parch', 'Fare', 'gender',
       'Pclass_1', 'Pclass_2', 'Pclass_3',  'Embarked_C', 'Embarked_Q',
       'Embarked_S',
       'Cabin_letter_A',
       'Cabin_letter_B', 'Cabin_letter_C', 'Cabin_letter_D', 'Cabin_letter_E',
       'Cabin_letter_F', 'Cabin_letter_G', 'Cabin_letter_T',
       'Cabin_letter_UNDIFINED','flag_women_childrens']



train_final = train_df[train_final_columns]

In [ ]:
from collections import Counter
def detect_outliers(df,features):
    outlier_indices = []
    
    for c in features:
        # 1st quartile
        Q1 = np.percentile(df[c],25)
        # 3rd quartile
        Q3 = np.percentile(df[c],75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5
        # detect outlier and their indeces
        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        # store indeces
        outlier_indices.extend(outlier_list_col)
    
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)
    
    return multiple_outliers

In [ ]:
train_final.loc[detect_outliers(train_final,["Age","SibSp","Parch","Fare"])]

,Survived,Pclass,Age,SibSp,Parch,Fare,gender,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Cabin_letter_A,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_T,Cabin_letter_UNDIFINED,flag_women_childrens
438,0,1,2.537273,1,4,4.439689,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
587,1,1,2.241737,1,1,0.887235,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
745,0,1,2.980578,1,1,0.728746,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
27,0,1,-0.787514,3,2,4.439689,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
88,1,1,-0.491977,3,2,4.439689,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
159,0,3,-1.507119,8,2,0.700721,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
180,0,3,-1.298185,8,2,0.700721,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
201,0,3,-1.507119,8,2,0.700721,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
324,0,3,-1.507119,8,2,0.700721,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
341,1,1,-0.418093,3,2,4.439689,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0


In [ ]:
train_final = train_final.drop(detect_outliers(train_final,["Age","SibSp","Parch","Fare"]),axis = 0).reset_index(drop = True)

In [ ]:
train_final

,Survived,Pclass,Age,SibSp,Parch,Fare,gender,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Cabin_letter_A,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_T,Cabin_letter_UNDIFINED,flag_women_childrens
0,0,3,-0.565861,1,0,-0.503402,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
1,1,1,0.616285,1,0,0.734222,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
2,1,3,-0.270325,0,0,-0.490356,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,1,0.394633,1,0,0.382778,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
4,0,3,0.394633,0,0,-0.487940,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,0,2,-0.196441,0,0,-0.392267,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
874,1,1,-0.787514,0,0,-0.063694,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
875,0,3,-0.544807,1,2,-0.190292,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
876,1,1,-0.270325,0,0,-0.063694,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(n_estimators=20,random_state=42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_final.drop("Survived",axis=1),
                                                    train_final.Survived, test_size=0.1, random_state=42)

In [ ]:
from sklearn.metrics import f1_score


In [ ]:
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_predict = model.predict(X_test)

display(confusion_matrix(y_predict,y_test))

display(f1_score(y_predict,y_test))

display(confusion_matrix( model.predict(X_train),y_train))
f1_score( model.predict(X_train),y_train)

array([[44,  7],
       [10, 29]])

0.7733333333333334

array([[491,  13],
       [  4, 293]])

0.9718076285240465

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

In [ ]:
regularization = keras.regularizers.l1_l2(l1=0.01, l2=0.01)
regularization_output = keras.regularizers.l2(0.01)

deep_model = keras.Sequential([layers.Dense(30, activation='tanh', input_shape=[X_train.shape[1],],bias_regularizer=regularization),])
deep_model.add(layers.Dropout(.2,))
deep_model.add(layers.Dense(30, activation='tanh',bias_regularizer=regularization))
deep_model.add(layers.Dropout(.2,))
deep_model.add(layers.Dense(15, activation='tanh',bias_regularizer=regularization))
deep_model.add(layers.Dropout(.2,))
deep_model.add(layers.Dense(10, activation='tanh',bias_regularizer=regularization))
deep_model.add(layers.Dropout(.1,))
deep_model.add(layers.Dense(5, activation='tanh',bias_regularizer=regularization))
deep_model.add(layers.Dropout(.1,))
deep_model.add(layers.Dense(1
                            ,activation='sigmoid'
                            ,bias_regularizer=regularization
                            ,activity_regularizer = regularization_output
                                ))

deep_model.compile(loss='binary_crossentropy', 
                   optimizer='adam'
                  #  optimizer = optimizer
                   )


In [ ]:
deep_model.fit(X_train, y_train, epochs=500,validation_data = (X_test,y_test), validation_split=0.1,verbose=1, batch_size=2**7)

Epoch 1/500
6/6 [==============================] - 0s 25ms/step - loss: 0.3757 - val_loss: 0.3355
Epoch 2/500
6/6 [==============================] - 0s 10ms/step - loss: 0.3784 - val_loss: 0.3412
Epoch 3/500
6/6 [==============================] - 0s 10ms/step - loss: 0.3906 - val_loss: 0.3498
Epoch 4/500
6/6 [==============================] - 0s 10ms/step - loss: 0.3802 - val_loss: 0.3494
Epoch 5/500
6/6 [==============================] - 0s 10ms/step - loss: 0.3788 - val_loss: 0.3398
Epoch 6/500
6/6 [==============================] - 0s 10ms/step - loss: 0.3850 - val_loss: 0.3277
Epoch 7/500
6/6 [==============================] - 0s 10ms/step - loss: 0.3823 - val_loss: 0.3285
Epoch 8/500
6/6 [==============================] - 0s 9ms/step - loss: 0.3667 - val_loss: 0.3292
Epoch 9/500
6/6 [==============================] - 0s 10ms/step - loss: 0.3684 - val_loss: 0.3306
Epoch 10/500
6/6 [==============================] - 0s 11ms/step - loss: 0.3794 - val_loss: 0.3385
Epoch 11/500
6/6 [==

In [ ]:
    deep_model.evaluate(X_test, y_test, batch_size=2**8)

1/1 [==============================] - 0s 15ms/step - loss: 0.4148


0.414825439453125

In [ ]:
def predict_deep_model(x):
  predict = deep_model.predict(x)
  return np.where(predict>0.5,1,0)

# predict_deep_model(X_train)

y_train.values
y_predict = predict_deep_model(X_test)

display(confusion_matrix(y_predict,y_test))

display(f1_score(y_predict,y_test))

display(confusion_matrix( predict_deep_model(X_train),y_train))
f1_score( predict_deep_model(X_train),y_train)

array([[48,  8],
       [ 6, 28]])

0.7999999999999999

array([[460,  66],
       [ 35, 240]])

0.8261617900172117

In [ ]:
mean_fare = np.mean(train_df.Fare)

mean_fare

-0.021091900968439086

In [ ]:
# test = test_df.copy()

# format_df(test)

train_final_columns

test_final = test [train_final_columns[1:]].copy()

test_final

,Pclass,Age,SibSp,Parch,Fare,gender,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Cabin_letter_A,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_T,Cabin_letter_UNDIFINED,flag_women_childrens
0,3,0.357691,0,0,-0.492208,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0
1,3,1.281243,1,0,-0.508234,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0
2,2,2.389505,0,0,-0.456291,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0
3,3,-0.196441,0,0,-0.476102,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0
4,3,-0.565861,1,1,-0.406039,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,-0.060948,0,0,-0.487940,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
414,1,0.690169,0,0,1.461271,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
415,3,0.653227,0,0,-0.503402,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
416,3,-0.060948,0,0,-0.487940,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0


In [ ]:
test_final.isna().sum()

test_final.Fare.fillna(mean_fare,inplace=True)

In [ ]:
test.shape


(418, 29)

In [ ]:
predict_test_final = predict_deep_model(test_final)

predict_dict = {'PassengerId': test.PassengerId.values,
                'Survived': predict_test_final.squeeze()}


predict_export = pd.DataFrame(predict_dict)

predict_path = '{}/{}'.format(path,'predict.csv')

predict_export.to_csv(predict_path,sep=',',index=False)

In [ ]:
rf_predict = model.predict(test_final)

predict_dict = {'PassengerId': test.PassengerId.values,
                'Survived': rf_predict.squeeze()}


predict_export = pd.DataFrame(predict_dict)

predict_path = '{}/{}'.format(path,'predict_rf.csv')

predict_export.to_csv(predict_path,sep=',',index=False)